# Persian Stance Classification - Deep Learning

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import numpy as np

# Mount Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# input files
cleaned_path = "/content/drive/MyDrive/Stance Detection Project/dataset cleaned/Clean_Claim_Body.csv"
train_path = "/content/drive/MyDrive/Stance Detection Project/dataset cleaned/train_data.csv"
test_path = "/content/drive/MyDrive/Stance Detection Project/dataset cleaned/test_data.csv"

stopwords_path = "/content/drive/MyDrive/Stance Detection Project/Baseline Zarharan/StopWords_fa.txt"

# output file
feature_file = '/content/drive/MyDrive/Stance Detection Project/Baseline Zarharan/features/ngrams.npy'

# Read Cleaned Data from CSV File

In [ ]:
import pandas as pd
dataset_clean = pd.read_csv(cleaned_path, index_col = 0, )

In [ ]:
clean_claim = dataset_clean['claim']
clean_body = dataset_clean['body']

In [ ]:
dataset_clean.head()

,claim,body,label
0,کلاهبرداری از رانندگان با شگرد نشت بنزین !,به گزارش خبرنگار گروه جامعه خبرگزاری میزان،29 ...,Discuss
1,تجاوز به دختر بازداشت شده و واژگونی ون گشت ارش...,انتشار کلیپ واژگونی ماشین گشت ارشاد توسط مردم ...,Discuss
2,تعظیم 20 دقیقه ای وزیر نیرو ژاپن به علت قطع بر...,وزیر نیروی ژاپن به علت قطع شدن برق؛ به همان مد...,Agree
3,سرمربیگری گاس هیدینک برای تراکتورسازی,به تازگی محمد تقوی استعفای خود را از سرمربیگری...,Discuss
4,کشف موجود عجیبی شبیه انسان در یک حفاری در پاکس...,پس از 20 سال حفاری با دقتی باورنکردنی، سرانجام...,Unrelated


# Read Data Train and Test from CSV Files

In [ ]:
data_train = pd.read_csv(train_path, index_col = 0, )
data_test = pd.read_csv(test_path, index_col = 0, )

In [ ]:
len(data_train), len(data_test)

(1597, 400)

# Three-grams Feature Extractor

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def char_3grams_5000_concat_all_data(headlines, bodies):

    def combine_head_and_body(headlines, bodies):
        return [headline + " " + body for i, (headline, body) in
                tqdm(enumerate(zip(headlines, bodies)))]

    # Load train data into CountVectorizer, get the resulting X-values and also the vocabulary
    # for the test data feature creation
    def get_features(headlines, bodies, headlines_all, bodies_all):
        # create vocab on basis of training data
        head_and_body = combine_head_and_body(headlines_all, bodies_all)
        head_and_body_tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 3), lowercase=True,
                                              max_features=5000, use_idf=False, norm='l2')
        head_and_body_tfidf.fit(head_and_body)
        vocab = head_and_body_tfidf.vocabulary_

        # create training feature vectors
        X_train_head_tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 3), lowercase=True,
                                             stop_words='english', vocabulary=vocab, use_idf=False, norm='l2')
        X_train_head = X_train_head_tfidf.fit_transform(headlines)

        X_train_body_tfidf = TfidfVectorizer(analyzer='char', ngram_range=(3, 3), lowercase=True,
                                             stop_words='english', vocabulary=vocab, use_idf=False, norm='l2')
        X_train_body = X_train_body_tfidf.fit_transform(bodies)

        X_train = np.concatenate([X_train_head.toarray(), X_train_body.toarray()], axis=1)

        return X_train

    h, b = data_train['claim'].tolist() , data_train['body'].tolist()
    h_test, b_test = data_test['claim'].tolist(), data_test['body'].tolist()

    # Comment out for clean ablation tests
    h.extend(h_test)
    b.extend(b_test)

    X_train = get_features(headlines, bodies, h, b)

    return X_train


In [ ]:
%%time

# extract 3grams features and save to file for future use
x = char_3grams_5000_concat_all_data(clean_claim, clean_body)

print(x.shape)

1997it [00:00, 511950.68it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:550: UserWarning: The parameter 'stop_words' will not be used since 'analyzer' != 'word'
  warnings.warn(


(1997, 10000)
CPU times: user 2.75 s, sys: 82.9 ms, total: 2.83 s
Wall time: 2.84 s


# Save Features

In [ ]:
np.save(feature_file, x)